In [5]:
from langchain_openai import OpenAI,ChatOpenAI

model=ChatOpenAI(
  model="qwen2.5-coder-1.5b-instruct",
  openai_api_key="EMPTY",
  base_url="http://127.0.0.1:1234/v1",
  temperature=0
)

In [3]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [13]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "用代数符号写出下面的方程，然后求解。使用格式\n\n方程：...\n解决方案：...\n\n"
    ),
    ("human", "{equation_statement}"),
])

# 2. 构建处理链
# 使用 RunnablePassthrough 将输入字符串映射到字典键 equation_statement
chain = (
    {"equation_statement": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

print(chain.invoke("x的3次方加7等于34"))

方程：\( x^3 + 7 = 34 \)

为了求解这个方程，我们首先将等式的两边减去7：

\[ x^3 + 7 - 7 = 34 - 7 \]
\[ x^3 = 27 \]

接下来，我们将等式的两边开立方根：

\[ \sqrt[3]{x^3} = \sqrt[3]{27} \]
\[ x = 3 \]

因此，解决方案是 \( x = 3 \)。


In [4]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField

model=model.configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temperature to use for the model",
    )
)

In [7]:
model.invoke("生成一个随机数，不要回答其他任何内容")

AIMessage(content='1234567890', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 29, 'total_tokens': 39, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2.5-coder-1.5b-instruct', 'system_fingerprint': 'qwen2.5-coder-1.5b-instruct', 'finish_reason': 'stop', 'logprobs': None}, id='run-1de394ad-e044-473d-981b-b58ebe075f78-0', usage_metadata={'input_tokens': 29, 'output_tokens': 10, 'total_tokens': 39, 'input_token_details': {}, 'output_token_details': {}})

In [9]:
model.invoke("生成一个随机数，不要回答其他任何内容")


AIMessage(content='1234567890', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 30, 'total_tokens': 40, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2.5-coder-1.5b-instruct', 'system_fingerprint': 'qwen2.5-coder-1.5b-instruct', 'finish_reason': 'stop', 'logprobs': None}, id='run-d5b68cfb-9c5e-4292-9b3f-a274e7b8dcf7-0', usage_metadata={'input_tokens': 30, 'output_tokens': 10, 'total_tokens': 40, 'input_token_details': {}, 'output_token_details': {}})

In [12]:
model.with_config(configurable={"llm_temperature":0.9}).invoke("生成一个随机数，不要回答其他任何内容")

AIMessage(content='1234567890', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 29, 'total_tokens': 39, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2.5-coder-1.5b-instruct', 'system_fingerprint': 'qwen2.5-coder-1.5b-instruct', 'finish_reason': 'stop', 'logprobs': None}, id='run-4ed39d45-6917-4b16-8cf9-2d642dfbbce1-0', usage_metadata={'input_tokens': 29, 'output_tokens': 10, 'total_tokens': 39, 'input_token_details': {}, 'output_token_details': {}})

@chain decorator
装饰器

In [16]:
model=ChatOpenAI(
  model="qwen2.5-coder-1.5b-instruct",
  openai_api_key="EMPTY",
  base_url="http://127.0.0.1:1234/v1",
  temperature=0.8
)

In [18]:
prompt1=ChatPromptTemplate.from_template("给我讲一个关于 {topic} 的故事")
prompt2=ChatPromptTemplate.from_template("{story}\n\n对上面这个故事进行修改，让故事变得更加口语化和幽默有趣")

In [20]:
from langchain_core.runnables import chain

@chain
def custom_chain(text):
    chain1=prompt1 | model | StrOutputParser()
    parsed_output1=chain1.invoke({"topic":text})
    
    chain2=prompt2 | model | StrOutputParser()
    return chain2.invoke({"story":parsed_output1})

In [21]:
custom_chain.invoke("关于动物的故事")

'从前，有一只活泼可爱的小兔子叫米米。她有着柔软的毛发，一双大眼睛，像星星一样闪亮，还有两只小耳朵，总是能听清一切声音。\n\n米米的家在一片草地上，那里绿意盎然，草地上的小溪叮咚作响。每到春天，小草就会破土而出，嫩绿的新叶长成，给大地披上了一层翠绿的纱衣。\n\n有一天，米米听说森林里有一只受伤的兔子，它叫乌龟。乌龟因为被猫抓伤而流血，它的伤口很难愈合。于是，米米决定去帮助乌龟。\n\n米米带着她的小铲子和一把小刀，踏上了前往森林的路。一路上，她遇到了很多困难：爬山、过河、涉水……但每一次都坚强地克服了。终于，在一阵阵风的吹拂下，她们到达了森林的尽头。\n\n在森林里，米米找到了一只受伤的乌龟，并用她的小铲子挖开了伤处。然后，她用刀切开伤口，帮助乌龟慢慢恢复。经过几天的治疗和照顾，乌龟渐渐康复了。\n\n当它看到米米时，它感激万分，紧紧地抱住米米，说：“谢谢你，我感谢你救了我的命。”\n\n从此以后，米米和乌龟成为了好朋友，它们一起玩耍，一起探险。每当森林里出现任何困难，米米都会毫不犹豫地伸出援手，帮助乌龟。她不仅教会了乌龟如何生存下来，还培养了它的好心和善良。\n\n这个故事告诉我们，无论遇到什么困难，只要我们勇敢面对，就一定能够克服它们。同时，我们也应该学会助人，为他人带来温暖和希望。'

In [33]:
@chain
def custom_chain2(text):
    chain=prompt1 | model | StrOutputParser() | {"story":RunnablePassthrough()} | prompt2 | model | StrOutputParser()
    return chain.invoke({"topic":text})

In [34]:
custom_chain2.invoke("有志者事竟成")

'《有志者事竟成》是一个充满正能量的故事。它讲述了一个年轻人叫林海天，他原本只是一个普通的工人，但凭借着自己的努力和毅力，在一次偶然的机会中获得了非常重要的机会。林海天利用这个机会，通过自己勤奋的学习和努力，成为了行业内的佼佼者。之后，林海天利用自己的技能和经验，成功地带领团队完成了多个大型项目。\n\n在这个故事中，我们看到了一个有志者所展现出的坚强毅力和决心。林海天在面对困难时没有退缩，而是选择了继续努力，最终取得了成功。这告诉我们，只要有决心和毅力，就没有什么是不可能的。\n\n同时，这个故事也告诉我们，无论遇到多大的挑战和困难，只要我们坚持不懈的努力和自我反省，就能够取得成功。这提醒我们要树立正确的价值观和人生观，不断学习和进步，为自己的未来打下坚实的基础。'